# ~ Category Arrangement Task Analysis ~

In [55]:
import os, glob
import pandas as pd
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
from scipy.spatial.distance import squareform
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [56]:
work_dir = '/Users/tuf69292/Google_Drive/categories_dimensions/subject_data'
os.chdir(work_dir)

## Import category arrangement data

In [68]:
subjects_data = []
for file in glob.glob("*.csv"):
    subjects_data.append(file)  

In [62]:
csv_subj = pd.read_csv('Meadows_Social_Relationship_Knowledge_v_v2_amazed-cub_2_table.csv').iloc[:, 1:]

r1 = relationships

## Create RDM of relationship categories for 1 subject

In [ ]:
rdm_r1 = pd.DataFrame(columns=relationships)

for r1_a in relationships:
    cols = [] 
    for col in csv_subj.columns:
        if r1_a in csv_subj[col].tolist():
            cols.append(col)
    #print cols
    for r1_b in relationships:
        if not cols:
            rdm_r1.loc[r1_a,r1_b] = 0
        elif csv_subj[cols].isin([r1_b]).any()[0]:
            rdm_r1.loc[r1_a,r1_b] = 1
        else:
            rdm_r1.loc[r1_a,r1_b] = 0
        

## Create RDM for all subjects

In [63]:
subject_cat_frames = {}
subject_flat_arrays = []
count = 1

for subj in subjects_data:
    csv_subj = pd.read_csv(subj).iloc[:, 1:]
    rdm_r1 = pd.DataFrame(columns=relationships)
    for rel_a in relationships:
        temp_relv_cols = []
        for col in csv_subj.columns:
            if rel_a in csv_subj[col].tolist():
                temp_relv_cols.append(col)
        for rel_b in relationships:
            if not temp_relv_cols:
                rdm_r1.loc[rel_a,rel_b] = 1
            elif csv_subj[temp_relv_cols].isin([rel_b]).any()[0]:
                rdm_r1.loc[rel_a,rel_b] = 0
            else:
                rdm_r1.loc[rel_a,rel_b] = 1
    rdm_r1 = rdm_r1.fillna(0)
    subject_cat_frames['subj_'+str(count)] = rdm_r1
    count=count+1


In [65]:
#Turn subject matrices to 1D arrays so they can be correlated
subject_flat_arrays = []
subjects_rdm = pd.DataFrame(columns=subject_cat_frames.keys())
for subj_a in subject_cat_frames.keys():
    subj_a_flat = subject_cat_frames[subj_a].values.flatten()
    subject_flat_arrays.append(subj_a_flat)
    for subj_b in subject_cat_frames.keys():
        subj_b_flat = subject_cat_frames[subj_b].values.flatten()
        subjects_rdm.loc[subj_a,subj_b] = np.corrcoef(subj_a_flat, subj_b_flat)[0][1]



## Calculate how often each of the relationships are grouped together

In [73]:
rel_gr_together = subject_cat_frames[list(subject_cat_frames)[0]]
for subj in list(subject_cat_frames)[1:]:
    #rel_gr_together.add(subject_cat_frames[subj_a], fill_value=0)
    rel_gr_together = rel_gr_together + subject_cat_frames[subj]
rel_gr_together = rel_gr_together.apply(pd.to_numeric)
rel_gr_together = rel_gr_together.div(len(subject_cat_frames))

In [74]:
rel_gr_together

,Wife - Husband,Nurse - Patient,Officer - Soldier,Neighbors,Athletic trainer - Trainee,Police officer - Offender,Principal - Teacher,Doctor - Nurse,Victim - Criminal,Coach - Athlete,...,Sorority sisters,Classmates,Club-member - Club-president,Political opponents,Friends-with-benefits,Foster-parent - Foster-child,Business rivals,Manager - Assistant,Victim - Witness,Business partners
Wife - Husband,0.000000,1.000000,1.000000,0.916667,1.000000,1.000000,1.000000,0.979167,1.000000,1.000000,...,0.812500,0.937500,1.000000,1.000000,0.458333,0.625000,1.000000,1.000000,0.979167,0.937500
Nurse - Patient,1.000000,0.000000,0.770833,0.937500,0.562500,0.875000,0.770833,0.583333,0.958333,0.750000,...,1.000000,0.937500,0.791667,0.958333,0.979167,0.916667,0.937500,0.812500,0.854167,0.833333
Officer - Soldier,0.979167,0.750000,0.000000,1.000000,0.604167,0.604167,0.250000,0.333333,0.770833,0.583333,...,1.000000,1.000000,0.625000,0.937500,1.000000,0.916667,0.895833,0.354167,0.833333,0.770833
Neighbors,0.895833,0.958333,0.979167,0.000000,0.958333,0.979167,0.958333,0.958333,0.958333,0.916667,...,0.520833,0.375000,0.833333,0.875000,0.729167,0.958333,0.875000,0.979167,0.750000,0.750000
Athletic trainer - Trainee,1.000000,0.562500,0.645833,0.937500,0.000000,0.895833,0.625000,0.583333,0.958333,0.375000,...,0.916667,0.958333,0.687500,0.916667,0.979167,0.979167,0.854167,0.750000,0.895833,0.770833
Police officer - Offender,0.979167,0.854167,0.625000,0.979167,0.875000,0.000000,0.791667,0.812500,0.229167,0.791667,...,1.000000,0.979167,0.833333,0.500000,0.979167,0.937500,0.583333,0.750000,0.750000,0.958333
Principal - Teacher,1.000000,0.770833,0.270833,0.958333,0.604167,0.791667,0.000000,0.270833,0.958333,0.541667,...,0.979167,0.895833,0.645833,0.937500,0.979167,0.958333,0.875000,0.416667,0.916667,0.729167
Doctor - Nurse,0.979167,0.625000,0.375000,0.958333,0.604167,0.812500,0.291667,0.000000,0.937500,0.625000,...,0.958333,0.916667,0.708333,0.937500,0.979167,0.958333,0.875000,0.541667,0.854167,0.583333
Victim - Criminal,0.979167,0.958333,0.791667,0.958333,0.958333,0.229167,0.958333,0.937500,0.000000,0.916667,...,1.000000,0.958333,0.916667,0.395833,0.979167,0.958333,0.479167,0.895833,0.645833,0.979167
Coach - Athlete,0.958333,0.750000,0.604167,0.916667,0.354167,0.791667,0.541667,0.604167,0.916667,0.000000,...,0.916667,0.916667,0.729167,0.875000,0.979167,0.937500,0.854167,0.666667,0.937500,0.833333


## View the representational dissimilarity matrix between all subjects

# ** Do not run cell below **

In [67]:
subjects_rdm

,subj_1,subj_2,subj_3,subj_4,subj_5,subj_6,subj_7,subj_8,subj_9,subj_10,...,subj_39,subj_40,subj_41,subj_42,subj_43,subj_44,subj_45,subj_46,subj_47,subj_48
subj_1,1,0.168303,0.242884,0.246024,0.2723,0.172421,0.21229,0.324263,0.2546,0.244418,...,0.285776,0.340895,0.303615,0.407036,0.21707,0.248502,0.279642,0.229904,0.335239,0.302243
subj_2,0.168303,1,0.159223,0.192502,0.135599,0.144967,0.157873,0.198326,0.146327,0.160177,...,0.176598,0.171434,0.178687,0.153038,0.137423,0.230435,0.165743,0.184932,0.211252,0.160767
subj_3,0.242884,0.159223,1,0.187592,0.16867,0.151554,0.152489,0.34359,0.334086,0.160372,...,0.235343,0.237609,0.261537,0.366442,0.224492,0.150908,0.303763,0.205441,0.376515,0.244933
subj_4,0.246024,0.192502,0.187592,1,0.232082,0.152954,0.331607,0.204017,0.244584,0.32852,...,0.351189,0.299642,0.274654,0.293909,0.301997,0.55876,0.29449,0.214769,0.324609,0.264037
subj_5,0.2723,0.135599,0.16867,0.232082,1,0.125976,0.238339,0.242811,0.222721,0.185606,...,0.235687,0.306097,0.238232,0.327629,0.221398,0.240969,0.232274,0.210864,0.259862,0.297154
subj_6,0.172421,0.144967,0.151554,0.152954,0.125976,1,0.246337,0.193629,0.196863,0.208003,...,0.182018,0.293926,0.271069,0.188943,0.208981,0.18158,0.224423,0.200392,0.168744,0.247403
subj_7,0.21229,0.157873,0.152489,0.331607,0.238339,0.246337,1,0.164832,0.202723,0.355565,...,0.281778,0.33324,0.240955,0.246938,0.230605,0.380997,0.221481,0.240785,0.243598,0.287665
subj_8,0.324263,0.198326,0.34359,0.204017,0.242811,0.193629,0.164832,1,0.351744,0.20042,...,0.278388,0.251398,0.366291,0.459288,0.245381,0.180478,0.303674,0.227929,0.342008,0.277311
subj_9,0.2546,0.146327,0.334086,0.244584,0.222721,0.196863,0.202723,0.351744,1,0.208341,...,0.279897,0.280884,0.320989,0.40185,0.311785,0.208815,0.249605,0.286463,0.381736,0.370296
subj_10,0.244418,0.160177,0.160372,0.32852,0.185606,0.208003,0.355565,0.20042,0.208341,1,...,0.312009,0.309105,0.281719,0.255529,0.310995,0.341002,0.277935,0.169515,0.239971,0.260946
